<a href="https://colab.research.google.com/github/sittidetw/dads5001_midterm_project/blob/main/DADS5001_Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sittidetw/dads5001_midterm_project.git

fatal: destination path 'dads5001_midterm_project' already exists and is not an empty directory.


In [2]:
pip install python-calamine

In [17]:
import pandas as pd
import numpy as np
from pathlib import Path

# Data Cleaning

In [18]:
import glob

path = "/content/dads5001_midterm_project/rd_testing_population/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
for file in files:
    print(file)

Total files: 7
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q1_20260215.xlsx


In [36]:
# Load the Excel file
df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
file_name = Path(file).name
file_period = file_name.split('_')
period = file_period[2]

df = df.transpose()
df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
#df.insert(loc=0, column='header', value=df.index)
#df

# Define conditions
conditions = [
    (df[0] == 'จังหวัด'),
    (df[0] == 'เขตสุขภาพ'),
    (df[0].str.contains('testing')) & (df[1].str.contains('Female')),
    (df[0].str.contains('testing')) & (df[1].str.contains('Male')),
    (df[0].str.contains('testing')) & (df[1].str.contains('All KP')),
    (df[0].str.contains('testing')) & (df[1].str.contains('Total')),
    (df[0].str.contains('Positive')) & (df[1].str.contains('Female')),
    (df[0].str.contains('Positive')) & (df[1].str.contains('Male')),
    (df[0].str.contains('Positive')) & (df[1].str.contains('All KP')),
    (df[0].str.contains('Positive')) & (df[1].str.contains('Total'))
]

# Define the corresponding values for each condition
values = ['province', 'health_region',
          'test_female', 'test_male', 'test_all_kp', 'test_total',
          'positive_female', 'positive_male', 'positive_all_kp', 'positive_total'
]

# Create the new column
df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
df['header'] = df['header'].str.replace('total_total','grand_total')
columns_to_drop = [0,1,2,'header_temp']
df = df.drop(columns=columns_to_drop)
df = df.transpose()
df.columns = df.loc['header'].values
df = df.drop(index='header')

# Add Period
df.insert(loc=0, column='quarter', value=period)
df.insert(loc=0, column='q', value=period[-2:])
df.insert(loc=0, column='year', value=period[:4])

# Change data type in dataframe
for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

#df

,year,q,quarter,province,health_region,test_male_fsw,test_male_migrant,test_male_msm,test_male_msw,test_male_partner_of_hiv,...,positive_female_msw,positive_female_partner_of_hiv,positive_female_prisoner,positive_female_pwid,positive_female_tg,positive_female_tgsw,positive_female_all_kps,positive_female_total,positive_all_kp_total,positive_grand_total
3,2025,Q1,2025Q1,เชียงใหม่,1,0,1,6147,317,25,...,0,2,0,0,0,0,4,33,79,154
4,2025,Q1,2025Q1,ลำพูน,1,0,0,982,0,2,...,0,1,1,0,0,0,2,7,6,16
5,2025,Q1,2025Q1,ลำปาง,1,0,0,323,3,4,...,0,3,0,0,0,0,3,9,21,34
6,2025,Q1,2025Q1,แพร่,1,0,0,1,1,0,...,0,0,0,0,0,0,0,6,1,19
7,2025,Q1,2025Q1,น่าน,1,0,0,12,0,2,...,0,0,0,0,0,0,0,3,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2025,Q1,2025Q1,พัทลุง,12,0,0,329,0,6,...,0,0,0,0,0,0,0,7,10,26
76,2025,Q1,2025Q1,ปัตตานี,12,0,0,178,0,0,...,0,0,0,0,0,0,0,1,4,13
77,2025,Q1,2025Q1,ยะลา,12,0,0,3,0,2,...,0,0,1,1,0,0,3,3,17,19
78,2025,Q1,2025Q1,นราธิวาส,12,0,0,3,3,0,...,0,1,0,0,0,0,1,5,3,13


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77 entries, 3 to 79
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   year                            77 non-null     string
 1   q                               77 non-null     string
 2   quarter                         77 non-null     string
 3   province                        77 non-null     string
 4   health_region                   77 non-null     string
 5   test_male_fsw                   77 non-null     int32 
 6   test_male_migrant               77 non-null     int32 
 7   test_male_msm                   77 non-null     int32 
 8   test_male_msw                   77 non-null     int32 
 9   test_male_partner_of_hiv        77 non-null     int32 
 10  test_male_prisoner              77 non-null     int32 
 11  test_male_pwid                  77 non-null     int32 
 12  test_male_tg                    77 non-null     int32 
 1

In [38]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
test_male_fsw,77.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
test_male_migrant,77.0,2.935065,11.532948,0.0,0.0,0.0,0.0,74.0
test_male_msm,77.0,713.376623,2413.346383,1.0,7.0,38.0,365.0,19720.0
test_male_msw,77.0,30.194805,166.643572,0.0,0.0,1.0,4.0,1424.0
test_male_partner_of_hiv,77.0,6.376623,13.393870,0.0,1.0,3.0,6.0,99.0
test_male_prisoner,77.0,208.077922,269.564647,0.0,1.0,157.0,259.0,1212.0
test_male_pwid,77.0,25.090909,73.460907,0.0,0.0,0.0,15.0,558.0
test_male_tg,77.0,38.727273,136.858132,0.0,0.0,1.0,21.0,1140.0
test_male_tgsw,77.0,6.272727,35.004374,0.0,0.0,0.0,0.0,249.0
test_male_all_kps,77.0,1031.051948,2804.763541,2.0,115.0,312.0,939.0,23162.0


In [39]:
df.columns

Index(['year', 'q', 'quarter', 'province', 'health_region', 'test_male_fsw',
       'test_male_migrant', 'test_male_msm', 'test_male_msw',
       'test_male_partner_of_hiv', 'test_male_prisoner', 'test_male_pwid',
       'test_male_tg', 'test_male_tgsw', 'test_male_all_kps',
       'test_male_total', 'test_female_fsw', 'test_female_migrant',
       'test_female_msm', 'test_female_msw', 'test_female_partner_of_hiv',
       'test_female_prisoner', 'test_female_pwid', 'test_female_tg',
       'test_female_tgsw', 'test_female_all_kps', 'test_female_total',
       'test_all_kp_total', 'test_grand_total', 'positive_male_fsw',
       'positive_male_migrant', 'positive_male_msm', 'positive_male_msw',
       'positive_male_partner_of_hiv', 'positive_male_prisoner',
       'positive_male_pwid', 'positive_male_tg', 'positive_male_tgsw',
       'positive_male_all_kps', 'positive_male_total', 'positive_female_fsw',
       'positive_female_migrant', 'positive_female_msm', 'positive_female_msw',
    